# 反爬：登入授權模擬

* 了解「登入權限機制」的反爬蟲機制
* 「登入權限機制」反爬蟲的因應策略

## 作業目標

* 找一個需要登入的網站試試看，並說明思考流程
（如果不知道要用哪個網站的話，可以試試看 https://github.com/new 網址，未登入時會被導向登入頁）



In [1]:
import requests
from bs4 import BeautifulSoup

# 於登入介面先取得token和cookies資訊
* 由於無法在'https://github.com/login' 直接登入，因此先儲存原先的token和cookies

In [2]:
response_login = requests.get('https://github.com/login')
soup = BeautifulSoup(response_login.text, 'lxml')
token = soup.find('input', {'name':'authenticity_token'})['value']
print('token：',token)

token： htbKt8wlFUU4YjBC4tTscX7zw9EQXkVaBPYRel7dK1ioWyu/7+HhIkui1RPfJSTewFOUT+LurWuJ2NY+7xV5sQ==


In [3]:
#print(soup.prettify())

In [3]:
cookies_login=response_login.cookies.get_dict()
print(cookies_login)

{'_octo': 'GH1.1.661938731.1615376113', 'logged_in': 'no', '_device_id': 'fe0df5b260bca46a31d51ec5cf7c90bf', '_gh_sess': 'RA5E8txe1WHD3TEoNfI%2FbipDGtt0Y%2Fi73RRDyniU%2B2rfuGRyMR0W%2F0PmAv3MCyUpyBpn%2FVvFBFd2pTpp3fjlvXe5MuZH9PRsXNY0k7amv5nAjZP3obVdOJofnLb8QXF0eB6xF6Oh8jf0Hh7rZgwpmL4ghrXoTvB0eLH%2BPGP4qJe4Nr5jFMju5szoKP2A4SCkZKHpH%2FG%2FQZhTquvtrWEXl9GvxoNai9XqvYNdeKLoK7Eo8gspq90Ac%2BFfDmizC4xArduJkRZQr86G5Hjubwpp2g%3D%3D--hIkWYoziLiTNfrCx--dNHN7Bv9kXVvv6e0%2Fkzc%2FA%3D%3D'}


# 至Github重新輸入的介面
* 發現輸入錯誤的帳密後將移至'https://github.com/session' 網頁，此網頁的headers將轉為post方法，且headers中多了Form Data。
  可以將Form Data覺得必要的值丟至data中，像是本身正確的帳密和token以及cookies，而token與cookies用剛剛取的，如果token與cookies是用
  session這個網頁的headers提供的，將無法登入。

In [5]:
D = {'commit':'Sign in',
     'authenticity_token':token,
     'login':#帳號#,
     'password':#密碼#
    }

response_session = requests.post('https://github.com/session',
                                data = D ,
                                cookies = cookies_login
                                )

cookies_session = response_session.cookies.get_dict()
print(cookies_session)

{'logged_in': 'yes', '__Host-user_session_same_site': 'uU4V7fLxTSwdmuxr_X440kY8InxIljfdyeTuhCzzvngw8QV3', '_gh_sess': '5SJ17ZXfoC1obhlYV08i9c3pb8J0N4cbKHL8QPoM3JT09P8ZK9LgwU8eeYbo9DqUkRrfIt3jVlOyczBXdG3%2F%2BvW5hVx1it2qH65RUlrMsECHCdtuOr03EgHldhvJOGOgWqinfWM5CJRiQfBq1QYjhzgf%2FZ5NjS8dl6pTcJhvMHYmx18GN9ApVUAPGE8xv69XgXF8QWYuVHtbq7ikUZ4bJhhCG63nzE0QwBcU7O%2FydIegYEIK6D1%2BDXCNa1zd5DNYtGQ5G2mbHw%2FjSsjhnojvpVYIZ0lyOGwQnnFlaF7movueO3ms--7VESDtPtEZMheRsH--WiHa7ztENoZ16uIQLkjfrg%3D%3D', 'has_recent_activity': '1', 'user_session': 'uU4V7fLxTSwdmuxr_X440kY8InxIljfdyeTuhCzzvngw8QV3'}


# logged_in: yes (成功登入)
* if用session網頁headers所提供的token和cookies，logged_in將為no

In [7]:
response = requests.get('https://github.com/', cookies = cookies_session)
soup = BeautifulSoup(response.text, 'lxml')
#print(soup.prettif())

In [8]:
print(soup.find_all('img')[-3])
print('\n')
print('login_account:',soup.find_all('img')[-3]['alt'])

<img alt="@andywu85127" class="avatar avatar-user" height="20" src="https://avatars.githubusercontent.com/u/78463626?s=40&amp;v=4" width="20"/>


login_account: @andywu85127
